# Notebook: View Dataset Statistics

This notebook is used to explore a given dataset.

## Packages

In [1]:
import pandas as pd
import tiktoken
import re

## Constants

In [2]:
DATASET_PATH = "reviews_dataset/balanced_reviews_sentences.csv"
LLM_MODEL_NAME = "gpt-3.5-turbo"

## Code

### General Statistics

In [3]:
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0.1,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,0,1906,875871390,8009090,16,Hausgemachte frische Pasta mit viel Auswahl un...,2023-01-21,Culture08862634821,"Dresden, Deutschland",Damit war die Sauce in Summe ein wenig zu salz...,4.0,dresden,Mamma Mia,de,Ein solides italienisches Restaurant in der Dr...,11,8f5a5889-8cce-40c4-a9d4-ec68c9ad3447,de
1,1,2443,911273338,1342313,1,Nicht nur überaus köstlich in romantischer Atm...,2023-08-18,Michael E,NaN,"Überragender Service, köstliche Speisen!Wir ko...",5.0,bielefeld,Jivino Enoteca,de,"Ungefragt wurde uns vorab serviert, was wir mö...",1,dfa4a582-b715-4fe5-ba4a-f143441b97bb,de
2,2,2192,877994419,1390048,1,Nicht für nur 2 Personen geeignet,2023-02-09,N_G_4000,"Fürth, Deutschland",Freundlichkeit und Qualität.Aber wenn Sie als ...,2.0,nürnberg,Bratwursthäusle,de,Im Garten bisher alles ok. Freundlichkeit und ...,1,57313614-0f5e-4855-80eb-f1fa735cdbb5,de
3,3,723,859195793,1190751,12,So lecker wie immer,2022-09-09,annaliebrern,NaN,Sie kochen für unsere Geburtstagsfeiern seit J...,5.0,frankfurt,Trattoria i Siciliani,de,Sie kochen für unsere Geburtstagsfeiern seit J...,0,80559f0d-8e77-4a6a-9970-5515bde2f0b8,de
4,4,563,850412704,718270,1,Lecker Kölsch,2022-07-28,kaiserdirk,"Leverkusen, Deutschland",Wir mögen Peters Kölsch.2.,4.0,köln,Peters Brauhaus,de,"Hier kehren wir immer wieder gerne ein, aus dr...",1,6089488a-38da-423e-8000-d8750df0dcf5,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,3495,3113,887337929,7236591,0,Sehr empfehlenswert,2023-04-23,Thomas R,"Baden-Württemberg, Deutschland",Man wurde freundlich bedient.,4.0,karlsruhe,Restaurant Terra Mare,de,Das Essen war einfach herrlich und eine riesig...,1,13aa2cca-5eef-4ece-a2f3-c6449950985e,de
3496,3496,3592,893275773,5207419,0,So gefällt Geburtstag feiern außer Haus,2023-06-03,Iris S,NaN,"Zum Servicepersonal, der Küche und dem „Chef i...",4.0,mönchengladbach,Brauerei zum Stefanus,de,Dem Anlass angemessen (Geburtstag) waren wir P...,6,0a0676a9-d275-4258-9398-12fb206cb5d7,de
3497,3497,550,888907029,718270,0,Lecker Bier und tolle Atmosphäre,2023-05-04,Axel S,"Braunschweig, Deutschland",Atmosphäre ist toll und man sollte in jedem Fa...,4.0,köln,Peters Brauhaus,de,Wir waren hier zum Abendessen. Atmosphäre ist ...,1,5a1f3ec9-8e3d-4b73-b341-856183ebe793,de
3498,3498,3175,889498235,7322855,4,Ein Reines Gedicht,2023-05-08,Tim R,NaN,Es schmeckt als ob die Kuh jeden Tag vom Bauer...,5.0,augsburg,SteakManufaktur,de,Zur Ambiente:Wir würden herzlich empfangen vom...,11,bf45e9b9-2317-4306-92f1-67d874158838,de


In [4]:
rating_counts = df['rating'].value_counts()
rating_counts

rating
4.0    700
5.0    700
2.0    700
3.0    700
1.0    700
Name: count, dtype: int64

In [5]:
rating_counts = df['city'].value_counts()
rating_counts

city
hamburg            375
leipzig            274
dresden            252
münchen            229
düsseldorf         171
frankfurt          161
stuttgart          156
köln               156
bonn               147
nürnberg           147
münster            135
augsburg           123
berlin             122
mannheim           120
bremen             116
bielefeld          112
mönchengladbach    103
bochum             103
wiesbaden           97
hannover            95
essen               93
karlsruhe           79
dortmund            57
wuppertal           46
duisburg            31
Name: count, dtype: int64

In [6]:
rating_counts = df['restaurant_name'].value_counts()
rating_counts

restaurant_name
Bullerei                    172
Ratskeller                  151
Mamma Mia                   137
Olive                        85
Schneider Bräuhaus           85
                           ... 
Argentina - Steak & Mehr      4
Tokyo Sushi Bar               3
Ola Lisboa                    3
EisLabor- Bonn Altstadt       3
Thai Banyan Restaurant        2
Name: count, Length: 121, dtype: int64

In [7]:
#df.groupby(['sentence_idx', 'review_id']).size().reset_index(name='count')

In [8]:
def count_words(text):
    words = re.findall(r'\w+', text)
    return len(words)

df['word_count'] = df['text'].apply(count_words)
average_word_count_by_rating = df.groupby('rating')['word_count'].mean()
average_word_count_by_rating

rating
1.0    15.648571
2.0    14.484286
3.0    13.367143
4.0    12.191429
5.0    11.661429
Name: word_count, dtype: float64

In [9]:
 df['word_count'].mean()

13.470571428571429

### Count Tokens (for LLM)

In [10]:
# Source: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_string(string: str, model_name: str) -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [11]:
df['llm_token_count'] = df['text'].apply(lambda x: num_tokens_from_string(x, LLM_MODEL_NAME))

In [12]:
pd.set_option('display.max_colwidth', None)

In [13]:
max_token_row = df[df['llm_token_count'] == df['llm_token_count'].max()]
print("Zeile mit den meisten Tokens:")
max_token_row#["llm_token_count"].values[0]

Zeile mit den meisten Tokens:


,Unnamed: 0.1,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language,word_count,llm_token_count
2856,2856,3122,887348853,908561,0,Willkommen in der Servicewüste „Anders am Turmberg“!,2023-04-23,Michael R,"Landau in der Pfalz, Deutschland","Der Chef steht mit Straßenkleidung in der Küche und lacht nur, über was war nicht klar – kein Wort des Bedauerns.Der Aperitif wurde nachgereicht und als Kompensation für ruiniertes Kleid und Handtasche zugegebenermaßen nicht berechnet.Schlechter Service scheint aber zum Ton des Hauses zu gehören, denn das Essen wurde serviert und erst danach erhielt man das Besteck direkt auf die Hand, Servietten lagen lieblos auf dem Tisch verstreut bzw. musste man anfordern und dafür zahlt man dann Spitzenpreise!Schaut man sich die Rezensionen über diese Lokalität auf den verschiedenen Foren an, so lebt die Kneipe fast nur von der Lage mit dem guten Blick auf Karlsruhe, aber dafür muss man Gottseidank nicht in das Lokal gehen!“ Im Übrigen hat auch uns fast jedes der dort monierten „Kleinigkeiten“ betroffen.Zusammengefasst ist dieses Etablissement nicht empfehlenswert und für den gebotenen Service zu teuer – Karlsruhe hat viele, deutlich bessere Lokale und unter gehobener Gastronomie stelle ich mir etwas anderes vor.",1.0,karlsruhe,Anders auf dem Turmberg,de,"2 volle Gläser mit Aperitif von einem Azubi über meine Frau geschüttet. So etwas kann ja mal passieren, auch wenn es in durchnässter Kleidung nicht mehr so schmeckt und zudem die Handtasche und der Sessel betroffen sind. Schlimm ist nur, dass weder eine richtige Entschuldigung kam, der Stuhl nicht gewechselt wurde und der Boden nicht gewischt wurde. Auch die Scherben wurden nicht alle entfernt. Hilfe oder Unterstützung, das Missgeschick zu beseitigen – Fehlanzeige.Lediglich der Hinweis, wir klären das später – toll, wenn einem keiner hilft, den Schaden zu begrenzen. Der Chef steht mit Straßenkleidung in der Küche und lacht nur, über was war nicht klar – kein Wort des Bedauerns.Der Aperitif wurde nachgereicht und als Kompensation für ruiniertes Kleid und Handtasche zugegebenermaßen nicht berechnet.Schlechter Service scheint aber zum Ton des Hauses zu gehören, denn das Essen wurde serviert und erst danach erhielt man das Besteck direkt auf die Hand, Servietten lagen lieblos auf dem Tisch verstreut bzw. musste man anfordern und dafür zahlt man dann Spitzenpreise!Schaut man sich die Rezensionen über diese Lokalität auf den verschiedenen Foren an, so lebt die Kneipe fast nur von der Lage mit dem guten Blick auf Karlsruhe, aber dafür muss man Gottseidank nicht in das Lokal gehen!“ Im Übrigen hat auch uns fast jedes der dort monierten „Kleinigkeiten“ betroffen.Zusammengefasst ist dieses Etablissement nicht empfehlenswert und für den gebotenen Service zu teuer – Karlsruhe hat viele, deutlich bessere Lokale und unter gehobener Gastronomie stelle ich mir etwas anderes vor.",5,60185e10-1715-4729-a483-cc2070f14615,de,157,279


In [14]:
with open("../prompt_template.txt", "r", encoding="utf-8") as file:
    prompt_template = file.read()

In [15]:
template_size = num_tokens_from_string(prompt_template, LLM_MODEL_NAME)
aspect_term_tag_size = num_tokens_from_string('<aspect-term aspect="ambient/interior" polarity="negative"></aspect-term>', LLM_MODEL_NAME)

In [16]:
template_size + 10 * max_token_row["llm_token_count"].values[0] + aspect_term_tag_size * 20

3394